In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GRBL


In [66]:
sys.path.append(os.path.abspath(".."))
from utils import picture

# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

> Jenga is played with 54 wooden blocks. Each block is three times longer than its width, and one fifth as thick as its length 1.5 cm × 2.5 cm × 7.5 cm (0.59 in × 0.98 in × 2.95 in).

https://en.wikipedia.org/wiki/Jenga

In [2]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]

In [3]:
# Jenga
BlockHeight = 25  # [mm]
BlockLength = 75  # [mm]

In [4]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 18  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [5]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),# Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick), # Even rows.
)
horizontal_brick_lines = np.linspace(0, BlockHeight, N_BrickRows, endpoint=False)

Lines parallel to the X-axis.
Separates rows of bricks.

In [6]:
horizontal_brick_lines

array([ 0.        ,  1.38888889,  2.77777778,  4.16666667,  5.55555556,
        6.94444444,  8.33333333,  9.72222222, 11.11111111, 12.5       ,
       13.88888889, 15.27777778, 16.66666667, 18.05555556, 19.44444444,
       20.83333333, 22.22222222, 23.61111111])

Lines parallel to the Y-axis.

In [7]:
vertical_brick_lines_tuple

(array([ 4.59401709,  9.18803419, 13.78205128, 18.37606838, 22.97008547,
        27.56410256, 32.15811966, 36.75213675, 41.34615385, 45.94017094,
        50.53418803, 55.12820513, 59.72222222, 64.31623932, 68.91025641,
        73.5042735 ]),
 array([ 2.29700855,  6.89102564, 11.48504274, 16.07905983, 20.67307692,
        25.26709402, 29.86111111, 34.45512821, 39.0491453 , 43.64316239,
        48.23717949, 52.83119658, 57.42521368, 62.01923077, 66.61324786,
        71.20726496]))

In [8]:
BlockBrick=GCode.Program()
BlockBrick.lines=list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    
    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(
        points=row_line_points,
    )
    
    BlockBrick.lines.append(line_)

In [9]:
for idx in range(1, len(horizontal_brick_lines)):

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]
    
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [vertical_brick_line, start_point_y],
                    [vertical_brick_line, end_point_y],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)
# Not drawing the last row of vertical lines.
idx+=1
vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]

start_point_y = horizontal_brick_lines[idx-1]
end_point_y = BlockHeight
        
for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
    transform = transform_tuple[np.mod(idx2, 2)]
    column_line_points = (
        np.array(
            [
                [vertical_brick_line, start_point_y],
                [vertical_brick_line, end_point_y],
            ]
        ),
    )
    column_line_points = np.matmul(transform, column_line_points)
    line_ = GCode.Line(column_line_points)
    BlockBrick.lines.append(line_)

In [10]:
BlockBrick.lines

[Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=75.0mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm, feed=300, power=150>,
 Line<len=1.38889mm

In [11]:
BlockBrick.time

2244.6480329307306

In [12]:
BlockBrick.dist

5494.295105861445

In [13]:
BlockBrick.generate_gcode()

In [110]:
BlockBrick

Program<lines=99>

In [14]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [15]:
cnc.status

'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [16]:
BlockBrick.machine=cnc

In [17]:
BlockBrick.run()

In [88]:
BlockBrick.lines[0]

Line<cut=75.0mm, F=300, S=150>

In [89]:
BlockBrick.lines[0].power

150

In [90]:
BlockBrick.lines[0].dynamic_power

True

In [ ]:
BlockBrick.run()